In [ ]:
# Experimental Topology
# lockin AO 2 -> Analog Attenuator Vdd
# lockin AO 3 -> Analog Attenuator Vc (Control Voltages)
# Keithley Output -> Voltage divider with 1 MOhm, DAQmx AI1
# AWG Sync Marker Out -> DAQmx PFI0
# AWG Samp. Marker Out -> PSPL Trigger

In [1]:
cd ..

C:\Users\qlab\Documents\GitHub\pycontrol


In [2]:
from pycontrol.filters.io import WriteToHDF5
import numpy as np
import matplotlib.pyplot as plt
from analysis.h5shell import h5shell

In [3]:
from experiments.Pulse_Switching_Search_Switch import SwitchSearchExperiment

In [4]:
exp = SwitchSearchExperiment()
exp.sample = "CSHE5-C2R3"
exp.field.value = 0.0081
exp.measure_current = 3e-6
exp.init_streams()
volts = np.arange(-0.7, -0.1, 0.05)
volts = np.append(volts, -1*np.flipud(volts))
volts = np.append(volts, np.flipud(volts))
durs = 1e-9*np.array([2,4])
exp.add_sweep(exp.pulse_voltage, volts)
exp.add_sweep(exp.pulse_duration, durs)

<SweptParameter: pulse_duration>

In [5]:
# Set up measurement network
wr = WriteToHDF5("data\CSHE-Switching\CSHE-Die5-C2R3\CSHE5-C2R3-Search_Switch_2016-07-26.h5")
edges = [(exp.daq_buffer, wr.data)]
exp.set_graph(edges)
exp.init_instruments()

In [6]:
exp.init_progressbar(num=2, notebook=True)
exp.run_sweeps()
exp.shutdown_instruments()

In [7]:
# Get data
f = h5shell(wr.filename,'r')
dset= f[f.grep('data')[-1]]
buffers = dset.value
f.close()

In [8]:
# Plot the result
buff_mean = np.mean(buffers, axis=(2,3))
figs = []
for res, dur in zip(buff_mean, durs):
    figs.append(plt.figure())
    plt.plot(volts, 1e-3*res/max(exp.measure_current,1e-7),'-o')
    plt.xlabel("AWG amplitude (V)", size=14);
    plt.ylabel("Resistance (kOhm)", size=14);
    plt.title("PSPL Switch Volt Search - {} - {}ns".format(exp.sample,dur*1e9))
plt.show()